## Tree
### typically Binary Tree
<b>Binary Tree</b>:   
a tree data structure in which each node has at most two children, which are referred to as the left child and the right child
<b>Binary Search Tree</b>:    
also called an ordered or sorted binary tree, is a rooted binary tree whose internal nodes each store a key greater than all the keys in the node's left subtree and less than those in its right subtree.

<b>Tree Traversal</b>: 
<b>inorder</b>: just suitable for binary tree, not suitable to be extended to N-ary tree
traverse the left subtree => visit the root node => traverse the right subtree  
root is in the middle => in  
<b>preorder</b>:  
visit the root node => traverse the left subtree => traverse the right subtree  
root is visited first => pre  
<b>postorder</b>:  
traverse the left subtree => traverse the right subtree => visit the root node  
root is visted last => post  
<b>level-order</b>:  
traverse the tree level by level  

<b>199.Binary Tree Right Side View </b>  
Keep track of level using the Len of list. So each level only add the right most value.    
If Len(list) == level, meaning loop into a new level, and can add a new value, which is the rightest value  
Traversal order is root, right, left, to make sure the rightest value of each level is first added.  

In [1]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

In [2]:
def rightSideView(root):
    def helper(node, result, level):
        if not node:
            return
        if len(result) == level:  # meaning entering into a new level of the tree
            result.append(node.val)
        helper(node.right, result, level+1)   # right first to ensure meet the requirement of uestion
        helper(node.left, result, level+1)
    result = []
    helper(root, result, 0)
    
    return result

<b>108.Convert Sorted Array to Binary Search Tree </b>  
still, inorder traversal: left, root, right, in this way the value is in ascending order.   
So first get the middle item of the ascending list, make it the root node. all the items in middle item’s left will be in the left tree part, all the items in the middle items’s right will be in the right tree part.  
<b>recursive</b>: 
<b>base case</b>: if array is empty, not valid, get none, otherwise keep getting middle item as root, and then make left tree, right tree  
Time complexity: O(N) since we visit each node exactly once.
Space complexity is the stack used for recursion, which for balanced tree should be O(logN).

In [3]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
def sortedArrayToBST(nums):
    if not nums:
        return None
    mid = len(nums)//2   # suppose 5 element, mid is 2, which is the third element, suppose 2 element, mid is 1, which is the second element
    node = TreeNode(nums[mid])
    node.left = sortedArrayToBST(nums[:mid])
    node.right = sortedArrayToBST(nums[mid+1:])
    return node

<b>95.Unique Binary Search Tree II </b>

way 1 is using recursive with binary search tree’s features
binary search tree feature: [1,n], if i is root, then its left part is composed of [1,i-1], right part is composed of [i+1, n]
recursive feature, now [1, i-1], [i+1, n] are the same Q as original
way 2 is recursive with binary search tree’s features plus dynamic programming

In [4]:
def generateTrees(n):
    """
    BST feature: [1,n], if i is root, then its left part is composed of [1,i-1], right part is composed of [i+1, n]
    DFS, recursive, now [1, i-1], [i+1, n] are the same Q as original
    """
    def helper(n_list):
        if not n_list:
            return [None]  # why []? => result is a list
        result = []
        for i in range(len(n_list)):
            for left in helper(n_list[:i]):
                for right in helper(n_list[i+1:]):
                    node = TreeNode(n_list[i])
                    node.left = left
                    node.right = right
                    result += [node]
        return result
    
    if n == 0:
        return []
    return helper([i for i in range(1, n+1)])
            

In [5]:
def generateTrees(n):
    """dynamic programming way!!"""
    result = [[]*(n+1)]  # save space, length from 0 to n
    if n ==0:
        return result[0]
    
    result[0].append(None)   # None is a valid scenario for cases with 0 element
    for length in range(1, n+1):  # range [1, n] how many lements in the tree
        for i in range(length):  # range[0, length-1]
            for left_tree in result[i]:  # left tree can have 0 to length-1 element
                for right_tree in result[length-1-i]:   # total number of element is length, -1 for the root, -i for the left tree, leftover is the number of elements in right tree
                    node = TreeNode(i+1)
                    node.left = left_tree
                    node.right = clone(right_tree, i+1)   # i+1 is the offset, result only save the structure
                    result[len].append(node)
    return result[n]

def clone(node, offset):
    if not node:
        return None
    new_node = TreeNode(node.val+offset)
    new_node.left = clone(node.left, offset)
    new_node.right = clone(node.right, offset)
    return new_node

<b>501.Find Mode in Binary Search Tree </b>  
For BST, if travel inorder, prev_node <= curr_node<=next_node. Compare curr node with prev node, if they match, increase curr_count (find a duplicate), if they dont match, reset curr_count to 1.  
have a global max_count var. compare max_count with curr_count in each traversal. If two counts match, that means find a result with same count, append value to result list. if curr_count > max_count, update max_count, reset result list to [new_node_val]   
remember to initiate the prev_node_val to None and update the prev_node_val with current node value in each traversal

In [7]:
def findMode(root):
    """
    utilizing Binary search tree feature
    if inorder, prev<=curr<=next node's value
    """
    prev=None
    curr_count = 0
    max_count = 0
    result = []
    
    def helper(node):
        if not node:
            return 
        helper(node.left)   # inorder traversal
        
        if node.val!=prev:
            curr_count=1   # A NEW START
        else:
            curr_count+=1
        if curr_count == max_count:
            result.append(node.val)  # multiple mode
        elif curr_count > max_count:
            result = [node.val]   # reset result list
            max_count = curr_count
        prev = node.val
        
        helper(node.right)
        
    helper(root)
    return result
            

<b>543.Diameter of Binary Tree </b>  
for each node: 
    get the longest path of the given node: get max_left, max_right, then max (history_max, max_left+max_right), update the history_max  
    get the longest single children path: get max_left, max_right,  max(max_left, max_right) + 1  
start point: if node is null, return 0

the helper function does two things  
1.calculate the longest child path of a given node
2.updating the outside class variable (the target var) while traversal.  
doing post order traversal, from bottom to top

In [8]:
def diameterOfBinaryTree(root):
    """
    DFS for everynode, calculate what its longest left and right children path
    post order traversal
    """
    max_legnth = 0
    def helper(node):
        if not node:
            return 0
        # post order traversal, from bottom to top
        left = helper(node.left)
        right = helper(node.right)
        
        # to get longest diameter path per given node
        max_legnth = max(max_length, left+right)
        
        # to get longest child path
        return max(left, right)+1   # +1 for the node itself
    
    helper(root)
    return max_length

<b>102.Binary Tree Level Order Traversal </b>  

key point is how to keep track of the level  
way 1: len(result) == level, then you are entering a new level  
way 2: two queues, one for current level, another keeps track of next level of nodes

In [9]:
def levelOrder(root):
    """DFS, using level and len of result to control where to add"""
    def helper(node, result, level):
        if not node:
            return
        if len(result) == level:   # meaning need to add a new [] to save that level
            result.append([node.val])
        else:
            result[level].append(node.val)
        helper(node.left, result, level+1)
        helper(node.right, result, level+1)
    result = []
    helper(root, result, 0)
    return result

In [10]:
from collections import deque
def levelOrder_2(root):
    """BFS O(N)"""
    queue = deque([(root, 0)])
    result = []
    while queue:
        node, level = queue.popleft()
        if len(result) == level:   # a new level should be added to the result
            result.append([])  # create a space in result to save that new level
        result[level].append(node.val)
        if node.left:
            queue.append((node.left, level+1))
        if node.right:
            queue.append((node.right, level+1))
    return result
            
    

In [11]:
def levelOrder_3(root):
    """BFS, but dont pop item from queue"""
    if not root:
        return []
    result = []
    queue = [root]
    while queue:
        next_queue = []
        result.append([])  # create a new space to save item from queue => current queue
        for node in queue:  # loop over current queue, which has nodes of the same levle
            result[-1].append(node.val)  # -1 locates to the last[] added to the result
            if node.left:
                next_queue.append(node.left)
            if node.right:
                next_queue.append(node.right)
        queue= next_queue   # update the queue, to go to next level of nodes
    return result
        
             
        

<b>Binary Tree Level Order Traversal II </b>   
way 1: this question can use the same methods above, but return the reversed result list
way 2: using special data structure to avoid reverse

In [ ]:
from collections import deque
def levelOrderBottom(root):
    if not root:
        return []
    queue = [root]
    result = deque()
    while queue:
        next_queue = []
        result.appendLeft([])  # add a new [] but to the left => so make latter level comes first
        for node in queue:
            result[0].append(node.val)   # here result[0] compare to result[-1]

<b> 124. binary tree maximum path sum </b>

ono def helper function does two things: one is to update the global var of max path sum, another is to return the current max branch value.

In [8]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:
    def __init__(self):
        self.max_value = float('-inf')
    def maxPathSum(self, root):
        
        def helper(node):
            """do two things"""
            if not node:
                return 0
            left = max(0, helper(node.left))  # if left branch's max value is negative, no need to add it
            right = max(0, helper(node.right))
            self.max_value = max(self.max_value, left+right+node.val)
            return max(left, right)+node.val

        helper(root)
        return self.max_value
    

In [9]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
a = Solution()
a.maxPathSum(root)

6

<b> 236. Lowest Common Ancestor of a binary tree </b>

so smart!  
If the current (sub)tree contains both p and q, then the function result is their LCA. If only one of them is in that subtree, then the result is that one of them. If neither are in that subtree, the result is None

In [10]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
def lowestCommonAncestor(root, p, q):
    # LCA can be the root node itself
    if root in (None,p, q):
        return root
    left = lowestCommonAncestor(root.left, p, q)
    right = lowestCommonAncestor(root.right, p, q)
    # left and right are all none empty, they must be p and q, the LCA found
    if left and right:
        return root
    # either p, q, or None
    else:
        return left or right

<b> 733 Flood hill </b>  

similar as the question: number of islands, but simpler. no need to loop the entire array, only need to start at the [sr][sc] point

In [11]:
def floodFill(image, sr, sc, newColor):
    """similar as number of islands question"""

    def helper(i, j):
        # index out of bound or 
        if i<0 or j<0 or i>=n or j>=m or image[i][j]!=color:
            return None
        image[i][j] = newColor
        helper(i+1, j)
        helper(i-1, j)
        helper(i, j+1)
        helper(i, j-1)

    n = len(image)
    if n == 0: return image
    m = len(image[0])
    if m == 0: return image
    color = image[sr][sc]
    if newColor == color: return image
    # only need to check from the point [sr][sc], no need to loop entire array
    helper(sr, sc)
    return image


In [12]:
image = [[1,1,1],[1,1,0],[1,0,1]]
sr = 1
sc = 1 
newColor = 2
floodFill(image, sr, sc, newColor)

[[2, 2, 2], [2, 2, 0], [2, 0, 1]]

<b> 98 validate binary search tree</b>  

<b>way one </b>  => remember this one  
using float(-inf) and float(inf) to initiate the range of value. if node's value is not within its corresponding range, then it is not a BST

In [13]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
def isValidBST(root):
    
    def helper(node, min_val, max_val):
        # node.val should be within the range [min_val, max_val]
        if not node:
            return True
        if node.val >= max_val or node.val <=min_val:
            return False
        return helper(node.left, min_val, node.val) and helper(node.right, node.val, max_val)
    
    return helper(root, float('-inf'), float('inf'))

In [15]:
root = TreeNode(2)
root.left = TreeNode(1)
root.right= TreeNode(3)
isValidBST(root)

True

<b> way two </b>  
same as above but using stack and while loop

In [17]:
def isValidBST(root):
    """
    if it is binary search tree, should 
    get left < root < right
    """

    if not root:
        return True
    stack = [(root, float('-inf'), float('inf'))]
    while stack:
        node, lower, higher = stack.pop()
        if not node:
            continue
        val = node.val
        # value not within the range to be a valid BST
        if val>=higher or val<=lower:
            return False
        else:
            stack.append((node.left, lower, val))
            stack.append((node.right, val, higher))
    return True

<b>way three</b>.  
inorder traversal, a bit harder to understand comparing with the way above

In [20]:
def isValidBST(root):
    stack, inorder = [], float('-inf')
    while stack or root:
        # go from top to bottom left
        while root:
            stack.append(root)
            root=root.left
        # first node is the very left bottom node, from there to the top
        root = stack.pop()
        if root.val <= inorder:
            return False
        inorder = root.val
        root=root.right
    return True

In [21]:
root = TreeNode(2)
root.left = TreeNode(1)
root.right= TreeNode(3)
isValidBST(root)

True

<b>101. Symmetric tree </b>

recursive call! have a helper function compare left and right.   
symmetric, so left's left vs right's right, left's right vs right's left

In [22]:
def isSymmetric(root):
    """left should == right"""
    def helper(left, right):
        # compare two nodes
        # all empty
        if (not left) and (not right):
            return True
        # all non-empty
        if left and right:
            # NOTE: symmetric, so left's left vs right's right, left's right vs right's left
            return (left.val==right.val) and helper(left.left, right.right) and helper(left.right, right.left)
        # one empty one is not
        return False

    if not root:
        return True
    return helper(root.left, root.right)

<b>103. binary tree zigzag level order traversal </b> 

very similar to question <b>102</b>  
here the way 1 corresponding to 102's way 3:

visit the nodes level by level, if the level is an odd level, revese the value  
how to update the next queue is the same as 102, breath first search. but how to save the value into result is where the zigzag happenning

In [ ]:
class Solution:
    def zigzagLevelOrder(root):
        if not root:
            return []
        result = []
        queue = [(root, 0)]
        while queue:
            next_queue = []
            for node, level in queue:
                if len(result)==level:
                    result.append([])
                result[level].append(node.val)
                if node.left:
                    next_queue.append((node.left, level+1))
                if node.right:
                    next_queue.append((node.right, level+1))
            if level%2==1:
                # from right to left
                result[level] = result[level][::-1]
            queue = next_queue
 
        return result

way 2 corresponding to <b>102</b>'s way 1, DFS way  

how to loop through each node is the same: DFS, preorder, node => left => right.   
how to save the result into the result list is different depending on what level the node is in. => where the deque.appendleft() is used


In [23]:
def zigzagLevelOrder(root):
    def helper(node, level):
        if not node:
            return
        if len(result) == level:
            result.append(deque([node.val]))
        else:
            if level%2==0:
                # result is from left to right
                result[level].append(node.val)
            else:
                # result should be from right to left
                result[level].appendleft(node.val)
        # the preorder traversal of DFS is not changed
        helper(node.left, level+1)
        helper(node.right, level+1)

    result = []  # a list of deque
    helper(root, 0)
    return result
        